In [1]:
from __future__ import print_function
import os
from io import BytesIO
import numpy as np
from functools import partial
import PIL.Image
import scipy.misc
import tensorflow as tf


![如何提高图像质量](./图像的高频低频解释.PNG)

In [5]:

def savearray(img_array,img_name):
    scipy.misc.toimage(img_array).save(img_name)
    print('img saved: %s' % img_name)

# 原始图像可能尺寸很大，从而导致内存耗尽
# 每次只对 tile_size * tile_size的大小图像计算梯度，避免内存耗尽
def calc_grad_tiled(img, t_grad, tile_size=512):
    sz = tile_size
    h, w = img.shape[:2]
    sx, sy = np.random.randint(sz, size=2)
    img_shift = np.roll(np.roll(img, sx, 1), sy, 0) 
    #先在行上整体移动，再在列上整体移动
    grad = np.zeros_like(img)
    for y in range(0, max(h-sz//2, sz), sz):
        for x in range(0, max(w-sz//2, sz), sz):
            sub = img_shift[y:y+sz, x:x+sz]
            g = sess.run(t_grad, {t_input:sub})
            grad[y:y+sz, x:x+sz] = g
    
    return np.roll(np.roll(grad, -sx, 1), -sy, 0)
    

def render_deepdream(t_obj,img0, iter_n=10, step=1.5, octave_n=4, octave_scale=1.4):
    t_score = tf.reduce_mean(t_obj)
    t_grad = tf.gradients(t_score, t_input)[0]
    img = img0.copy()
    
    # 将图像进行金字塔分解，从而分为高鹏部分和低频部分
    # level 0 表示原图像
    
    octaves = []
    
    for i in range(octave_n - 1):
        # 图像高和宽的元组
        hw = img.shape[:2] 
        # 对图像分解，除以octav_scale进行缩小，再resize 重设为何原图一样大的图像
        lo = resize(img, np.int32(np.float32(hw)/octave_scale))
        # 用原图 减去 低频成分，得到高频成分
        hi = img - resize(lo, hw)
        img = lo
        # 保存高频成分到金字塔中
        octaves.append(hi)
    
    # 首先生成低频图像，再依次放大并加上高频
    for octave in range(octave_n):
        if octave > 0:
            hi = octaves[-octave]
            img = resize(img, hi.shape[:2]) + hi
        for i in range(iter_n):
            # 避免内存耗尽
            g = calc_grad_tiled(img, t_grad)
            img += g * (step / (np.abs(g).mean() + 1e-7))
    
    img = img.clip(0,255)
    filename = 'render_deepdream.jpg'
    savearray(img, filename)
    im = PIL.Image.open(filename).show()

# 将图像放大ratio倍:
def resize_ratio(img, ratio):
    min = img.min()
    max = img.max()
    # 归一化
    img = (img-min) / (max-min)  * 255
    # img resize
    img = np.float32(scipy.misc.imresize(img, ratio))
    # 还原归一化前的数值
    img= img/255 * (max-min) + min
    return img

# 调整图像尺寸   和上面的一模一样？？
def resize(img, hw):
    min = img.min()
    max = img.max()
    # 归一化
    img = (img-min) / (max-min)  * 255
    # img resize
    img = np.float32(scipy.misc.imresize(img, hw))
    # 还原归一化前的数值
    img= img/255 * (max-min) + min
    return img

    

In [3]:

graph = tf.Graph()
model_fn = 'tensorflow_inception_graph.pb'
sess = tf.InteractiveSession(graph = graph)
with tf.gfile.FastGFile(model_fn, 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

t_input = tf.placeholder(np.float32, name='input')
imagenet_mean = 117.0
t_preprocessed = tf.expand_dims(t_input - imagenet_mean ,0)
tf.import_graph_def(graph_def, {'input': t_preprocessed})

In [6]:
name = 'mixed4c'
layer_output = graph.get_tensor_by_name('import/%s:0' % name)

img0 = PIL.Image.open('car.jpeg')
img0 = np.float32(img0)
render_deepdream(tf.square(layer_output), img0)


img saved: render_deepdream.jpg


In [8]:
img0 = PIL.Image.open('wwx.jpg')
img0 = np.float32(img0)
render_deepdream(tf.square(layer_output), img0)


img saved: render_deepdream.jpg
